In [ ]:
import numpy as np

# Sigmoid and Softmax functions
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    exp_vals = np.exp(x - np.max(x, axis=1, keepdims=True))  # for numerical stability
    return exp_vals / np.sum(exp_vals, axis=1, keepdims=True)

# Initialize parameters
def initialize_parameters(input_nodes, hidden_nodes, output_nodes):
    hidden_weights = np.random.randn(input_nodes, hidden_nodes)
    hidden_biases = np.zeros((1, hidden_nodes))
    output_weights = np.random.randn(hidden_nodes, output_nodes)
    output_biases = np.zeros((1, output_nodes))
    return hidden_weights, hidden_biases, output_weights, output_biases

# Forward propagation
def forward_propagation(board, hidden_weights, hidden_biases, output_weights, output_biases):
    hidden_layer = sigmoid(np.dot(board, hidden_weights) + hidden_biases)
    output_layer = softmax(np.dot(hidden_layer, output_weights) + output_biases)
    return hidden_layer, output_layer

# Backpropagation
def backpropagation(board, hidden_layer, output_layer, target, hidden_weights, hidden_biases, output_weights, output_biases, learning_rate=0.1):
    output_error = output_layer - target
    output_delta = output_error
    hidden_error = np.dot(output_delta, output_weights.T)
    hidden_delta = hidden_error * hidden_layer * (1 - hidden_layer)

    output_weights -= np.dot(hidden_layer.T, output_delta) * learning_rate
    output_biases -= np.sum(output_delta, axis=0, keepdims=True) * learning_rate

    hidden_weights -= np.dot(board.T, hidden_delta) * learning_rate
    hidden_biases -= np.sum(hidden_delta, axis=0) * learning_rate

# Function to select a move based on probabilities
def select_move(probabilities, available_moves):
    valid_probs = [probabilities[0][move] for move in available_moves]
    selected_move = available_moves[np.argmax(valid_probs)]
    return selected_move

# Check game status
def check_game_status(board):
    for player in [-1, 1]:
        for line in [[0, 1, 2], [3, 4, 5], [6, 7, 8], [0, 3, 6], [1, 4, 7], [2, 5, 8], [0, 4, 8], [2, 4, 6]]:
            if board[line[0]] == player and board[line[1]] == player and board[line[2]] == player:
                return player
    if 0 not in board:
        return 0  # Draw
    return None

# Function to update the board based on the move
def update_board(board, move, player):
    board[move] = player
    return board

# Display the current board
def display_board(board):
    symbols = {0: ' ', 1: 'X', -1: 'O'}
    for i in range(3):
        print(f"| {symbols[board[3*i]]} | {symbols[board[3*i + 1]]} | {symbols[board[3*i + 2]]} |")
    print("-------------")

# Define neural network architecture and initialize parameters
input_nodes = 9
hidden_nodes = 100
output_nodes = 9

hidden_weights, hidden_biases, output_weights, output_biases = initialize_parameters(input_nodes, hidden_nodes, output_nodes)

# Stopping criterion (for demonstration, based on a fixed number of iterations)
max_iterations = 1000
iteration = 0

# Training Loop
while iteration < max_iterations:
    board = np.zeros(9, dtype=int)  # Initialize board
    game_over = False
    current_player = 1

    while not game_over:
        display_board(board)
        print(f"Player {current_player}'s turn.")

        # Forward propagation
        hidden_layer, output_layer = forward_propagation(board.reshape(1, 9), hidden_weights, hidden_biases, output_weights, output_biases)

        # Make a move based on probabilities
        available_moves = [i for i in range(9) if board[i] == 0]
        selected_move = select_move(output_layer, available_moves)

        # Update board and check game status
        board = update_board(board, selected_move, current_player)
        game_status = check_game_status(board)

        if game_status is not None:
            game_over = True
            if game_status == 0:
                print("It's a draw!")
            else:
                print(f"Player {game_status} wins!")
        else:
            current_player *= -1  # Switch player

    iteration += 1
